# Retrieval model: item-to-item fine-tuned




Since our first item-item Retrieval model showed significant results on the train and test data, we would like to further fine-tine this model as the baseee one has shown signs of an overfit. The test results were by 30% less in accuracy rate on Top-10 in comparison to the train data. 

In order to address the overfit, we will add Dropout layer of 20% of our data and add a Dense layer with relu activation. 

### Imports

In [1]:
! pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install -q tensorflow-recommenders
! pip install -q --upgrade tensorflow-datasets
! pip install -q scann

     |████████████████████████████████| 89 kB 3.7 MB/s 
     |████████████████████████████████| 4.7 MB 4.8 MB/s 
     |████████████████████████████████| 10.4 MB 4.7 MB/s 
     |████████████████████████████████| 578.0 MB 13 kB/s 
     |████████████████████████████████| 1.7 MB 75.7 MB/s 
     |████████████████████████████████| 5.9 MB 64.8 MB/s 
     |████████████████████████████████| 438 kB 81.7 MB/s 


In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_recommenders as tfrs

# import interactive table 
from google.colab import data_table
data_table.enable_dataframe_formatter()

# set seed
tf.random.set_seed(42)

### Preparing the dataset

In [4]:
# mount G-Drive and load data
from google.colab import drive
drive.mount('/content/drive')

# load data subset 
gdrive_path = '/content/drive/MyDrive/ModelingData'
path = os.path.join(gdrive_path, "ratings")

ratings = tf.data.Dataset.load(path)

Mounted at /content/drive


In [5]:
# Select the basic features.

products = ratings.map(lambda x: x['data']['product_title'])

In [6]:
# train-test split
tf.random.set_seed(42)
shuffled = ratings.shuffle(92_096, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(92_096)
test = shuffled.skip(92_096).take(23_024)

In [7]:
# vocabulary to map raw feature values to embedding vectors
product_titles = products.batch(50_000)
unique_product_titles = np.unique(np.concatenate(list(product_titles)))

unique_product_titles[:10]

array([b'! Set 7 Colors Small S Replacement Bands + 1pc Free Small Grey Band With Clasp for Fitbit FLEX Only /No tracker/ 1pc Teal (Blue/Grey) 1pc Purple / Pink 1pc Red (Tangerine) 1pc Green 1pc Slate (Blue/Grey) 1pc Black 1pc Navy (Blue) Bands Wireless Activity Bracelet Sport Wristband Fit Bit Flex Bracelet Sport Arm Band Armband',
       b'! Small S 1pc Green 1pc Teal (Blue/Green) 1pc Red (Tangerine) Replacement Bands + 1pc Free Small Grey Band With Clasp for Fitbit FLEX Only /No tracker/ Wireless Activity Bracelet Sport Wristband Fit Bit Flex Bracelet Sport Arm Band Armband',
       b'! Small S 1pc Teal (Blue/Green) 1pc Purple / Pink Replacement Bands + 1pc Free Small Grey Band With Clasp for Fitbit FLEX Only /No tracker/ Wireless Activity Bracelet Sport Wristband Fit Bit Flex Bracelet Sport Arm Band Armband',
       b'"""SEASON SPECIAL"""THE ORIGINAL HEAVY DUTY BIG GRIZZLY COT-HEAVY DUTY QUALITY w/ IPHONE Holder & Drink Holder-High Quality Product-10 YEARS WARRANTY-84\xe2\x80\x9d L

In [8]:
# dimensionality of the query and candidate representations:
embedding_dimension = 64

### Implementing the model

In [9]:
# create product model to be used as both query and candidate models with Dropout rate of 20% and additional layer of relu activation. 
product_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_product_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_product_titles) + 1, embedding_dimension),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(32, activation = 'relu')
])

In [47]:
# define metric
metrics = tfrs.metrics.FactorizedTopK(
  candidates=products.batch(128).map(product_model)
)

In [11]:
# define task
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [49]:
# create a model based on TensorFlow Recommenders Model class
class AmazonModel(tfrs.Model):

  def __init__(self, user_model, product_model):
    super().__init__()
    self.product_model: tf.keras.Model = product_model
    self.user_model: tf.keras.Model = product_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    product_embeddings = self.product_model(features['data']["product_title"])
    # And pick out the product features and pass them into the product model,
    # getting embeddings back.
    positive_product_embeddings = self.product_model(features['data']["product_title"])

    # The task computes the loss and the metrics.
    return self.task(product_embeddings, positive_product_embeddings)

In [50]:
# initiate model
item_item_model_2 = AmazonModel(product_model, product_model)
item_item_model_2.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

### Fitting and Evaluating the model

In [14]:
# shuffle, batch, and cache train and test data
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [15]:
# train the model
item_item_model_2.fit(cached_train, epochs = 5)

Epoch 1/5
12/12 [==============================] - 134s 10s/step - factorized_top_k/top_1_categorical_accuracy: 0.0999 - factorized_top_k/top_5_categorical_accuracy: 0.1012 - factorized_top_k/top_10_categorical_accuracy: 0.1111 - factorized_top_k/top_50_categorical_accuracy: 0.1739 - factorized_top_k/top_100_categorical_accuracy: 0.2065 - loss: 49788.5066 - regularization_loss: 0.0000e+00 - total_loss: 49788.5066
Epoch 2/5
12/12 [==============================] - 126s 11s/step - factorized_top_k/top_1_categorical_accuracy: 0.2515 - factorized_top_k/top_5_categorical_accuracy: 0.2935 - factorized_top_k/top_10_categorical_accuracy: 0.3544 - factorized_top_k/top_50_categorical_accuracy: 0.5362 - factorized_top_k/top_100_categorical_accuracy: 0.5805 - loss: 28180.6046 - regularization_loss: 0.0000e+00 - total_loss: 28180.6046
Epoch 3/5
12/12 [==============================] - 123s 10s/step - factorized_top_k/top_1_categorical_accuracy: 0.3269 - factorized_top_k/top_5_categorical_accuracy: 

In [16]:
# evaluate model
item_item_model_2.evaluate(cached_test, return_dict=True)

6/6 [==============================] - 45s 7s/step - factorized_top_k/top_1_categorical_accuracy: 0.3628 - factorized_top_k/top_5_categorical_accuracy: 0.4931 - factorized_top_k/top_10_categorical_accuracy: 0.5757 - factorized_top_k/top_50_categorical_accuracy: 0.6792 - factorized_top_k/top_100_categorical_accuracy: 0.6987 - loss: 10905.0732 - regularization_loss: 0.0000e+00 - total_loss: 10905.0732


{'factorized_top_k/top_1_categorical_accuracy': 0.36275190114974976,
 'factorized_top_k/top_5_categorical_accuracy': 0.4930507242679596,
 'factorized_top_k/top_10_categorical_accuracy': 0.5757470726966858,
 'factorized_top_k/top_50_categorical_accuracy': 0.6792477369308472,
 'factorized_top_k/top_100_categorical_accuracy': 0.6987491250038147,
 'loss': 7184.37353515625,
 'regularization_loss': 0,
 'total_loss': 7184.37353515625}

Fine-tuned Item-to-item model is performing better in comparison to item-to-item model. The overfitting is not as significant as it was. The training accuracy for Top-10 is 69% vs 57% on testing. Our current model also performs much better on Top-1 categorical accuracy: 36% on the test vs 20.5% on the train.

### Serving and saving the model

In [31]:
# recommending Top-10 products for customer 52228204

# Create a item_item_model_2 that takes in raw query features, and
brute_force = tfrs.layers.factorized_top_k.BruteForce(item_item_model_2.product_model)
# recommends products out of the entire products dataset.
brute_force.index_from_dataset(
  tf.data.Dataset.zip((products.batch(100), products.batch(100).map(item_item_model_2.product_model)))
)

# Get recommendations.
_, titles = brute_force(tf.constant(["52228204"]))
print(f"Recommendations for user 52228204: {titles[0, :10]}")

Recommendations for user 52228204: [b'Orikaso Superlight, Ultracompact Folding Cup / Tasse - Green'
 b'SKS Velo 65 MTB Snap-On Bicycle Fenders - Pair'
 b'Huffy Metrixx 20" Freestyle Bicycle (EA)'
 b"\xc2\xa0UltraClub Men's Long-Sleeve Cypress Denim with Pocket (Navy) (Medium)"
 b'Manzella Mens Stalker Hunting Gloves'
 b'Oakley Frogskins Aquatique Collection Adult Sunglasses - Abyss / Positive Red / One Size Fits All - LIMITED EDITION (24-358)'
 b'Smartwool Hike Liner Crew Socks' b'Smartwool Hike Liner Crew Socks'
 b'Eureka N!ergy 12-Foot by-10-Foot Eight-Person Family Tent'
 b"Pearl Izumi Women's Sugar Printed Shorts"]


In [32]:
# time BruteForce retrieval execusion. 
%timeit _, titles = brute_force(tf.constant(["52228204"]), k=10)

3.03 ms ± 78.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


There is still some repetition of recommendations, but not as extreme as in the basemodel. 

In [33]:
# model serving: saving the model to G-Drive

# Export the query model.
gdrive_path = '/content/drive/MyDrive/Models'
path = os.path.join(gdrive_path, "item_item_model_2")

# Save the brute_force.
tf.saved_model.save(brute_force, path)

# Load it back; can also be done in TensorFlow Serving.
item_item_model_3 = tf.saved_model.load(path)

# Pass a user id in, get top predicted movie titles back.
scores, titles = item_item_model_3(["52228204"])

print(f"Recommendations: {titles[0][:3]}")


Recommendations: [b'Avenir Hartigan Helmet, White, Large/X-Large/59-62-cm'
 b'Italy "Italia" Vintage Flag International Mens T-Shirt #1180'
 b'Hot Headz Polarex Hunting Cap with Ear Flaps, Orange, One Size']


Adding ScaNN layer for quick retrieval and saving it to G-Drive. 

In [33]:
# adding ScaNN layer
scann = tfrs.layers.factorized_top_k.ScaNN(item_item_model_2.product_model)
scann.index_from_dataset(
  tf.data.Dataset.zip((products.batch(100), products.batch(100).map(item_item_model_2.product_model)))
)

# Get recommendations.
_, titles = scann(tf.constant(["52228204"]))
print(f"Recommendations for user 52228204: {titles[0, :10]}")

Recommendations for user 52228204: [b'Pilo D92 Black Derailleur Hanger - Fits: Iron Horse Downhill SGS DH WC/ Team/ PRO'
 b'Bandcase Replacement Vivid Color Wristband with Clasp for Fitbit Flex Activity & Sleep Tracker (No Tracker) (Orange, Small)'
 b'Avenir Ginger Youth Bike Helmet' b'DAKINE Ava Shoulder Bag - jasmine'
 b'Sinland Microfiber Ultra Compact Fast Drying Absorbent Travel Sports Towels(Purple, 32"x60")'
 b'Huffy Metrixx 20" Freestyle Bicycle (EA)'
 b'BESTOPE High Quality & Brand New Vintage Retro Floral Ladies Canvas School Backpack'
 b'Manzella Mens Stalker Hunting Gloves' b'Lucky Crew Complete Scooter'
 b'Doomagic N17 Foldable Backpack Packable Handy Waterproof Bag Lightweight For Travel Camping Hiking Daypack (Blue)']


In [37]:
#timeit
%timeit _, titles = scann(tf.constant(["52228204"]))

3.25 ms ± 30.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [38]:
# exporting ScaNN layer

# Export the query model.
gdrive_path = '/content/drive/MyDrive/Models'
path = os.path.join(gdrive_path, "item_item_model_2")

# Save the scann.
tf.saved_model.save(
    scann,
    path,
    options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"])
)

# Load it back; can also be done in TensorFlow Serving.
item_item_model_3 = tf.saved_model.load(path)

# Pass a user id in, get top predicted movie titles back.
scores, titles = item_item_model_3(["52228204"])

print(f"Recommendations: {titles[0][:10]}")

Recommendations: [b'Pilo D92 Black Derailleur Hanger - Fits: Iron Horse Downhill SGS DH WC/ Team/ PRO'
 b'Bandcase Replacement Vivid Color Wristband with Clasp for Fitbit Flex Activity & Sleep Tracker (No Tracker) (Orange, Small)'
 b'Avenir Ginger Youth Bike Helmet' b'DAKINE Ava Shoulder Bag - jasmine'
 b'Sinland Microfiber Ultra Compact Fast Drying Absorbent Travel Sports Towels(Purple, 32"x60")'
 b'Huffy Metrixx 20" Freestyle Bicycle (EA)'
 b'BESTOPE High Quality & Brand New Vintage Retro Floral Ladies Canvas School Backpack'
 b'Manzella Mens Stalker Hunting Gloves' b'Lucky Crew Complete Scooter'
 b'Doomagic N17 Foldable Backpack Packable Handy Waterproof Bag Lightweight For Travel Camping Hiking Daypack (Blue)']


## model 4

Adding additional Dropout layer of 20% to test if model performance will improve.

In [37]:
product_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_product_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_product_titles) + 1, embedding_dimension),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(32, activation = 'relu'), 
  tf.keras.layers.Dropout(0.2)
])

In [38]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=products.batch(128).map(product_model)
)

In [39]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [40]:
class AmazonModel(tfrs.Model):

  def __init__(self, user_model, product_model):
    super().__init__()
    self.product_model: tf.keras.Model = product_model
    self.user_model: tf.keras.Model = product_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    product_embeddings = self.product_model(features['data']["product_title"])
    # And pick out the product features and pass them into the product model,
    # getting embeddings back.
    positive_product_embeddings = self.product_model(features['data']["product_title"])

    # The task computes the loss and the metrics.
    return self.task(product_embeddings, positive_product_embeddings)

In [41]:
# initiate model
item_item_model_4 = AmazonModel(product_model, product_model)
item_item_model_4.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [42]:
# shuffle, batch, and cache train and test data
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [43]:
# train the model
item_item_model_4.fit(cached_train, epochs = 5)

Epoch 1/5
12/12 [==============================] - 187s 15s/step - factorized_top_k/top_1_categorical_accuracy: 0.0958 - factorized_top_k/top_5_categorical_accuracy: 0.0971 - factorized_top_k/top_10_categorical_accuracy: 0.1086 - factorized_top_k/top_50_categorical_accuracy: 0.1708 - factorized_top_k/top_100_categorical_accuracy: 0.2029 - loss: 50051.5341 - regularization_loss: 0.0000e+00 - total_loss: 50051.5341
Epoch 2/5
12/12 [==============================] - 181s 15s/step - factorized_top_k/top_1_categorical_accuracy: 0.2440 - factorized_top_k/top_5_categorical_accuracy: 0.2832 - factorized_top_k/top_10_categorical_accuracy: 0.3437 - factorized_top_k/top_50_categorical_accuracy: 0.5330 - factorized_top_k/top_100_categorical_accuracy: 0.5773 - loss: 28424.0411 - regularization_loss: 0.0000e+00 - total_loss: 28424.0411
Epoch 3/5
12/12 [==============================] - 178s 15s/step - factorized_top_k/top_1_categorical_accuracy: 0.3148 - factorized_top_k/top_5_categorical_accuracy: 

In [45]:
# evaluate model
item_item_model_4.evaluate(cached_test, return_dict=True)

6/6 [==============================] - 53s 9s/step - factorized_top_k/top_1_categorical_accuracy: 0.4896 - factorized_top_k/top_5_categorical_accuracy: 0.5724 - factorized_top_k/top_10_categorical_accuracy: 0.6232 - factorized_top_k/top_50_categorical_accuracy: 0.6944 - factorized_top_k/top_100_categorical_accuracy: 0.7077 - loss: 11046.2619 - regularization_loss: 0.0000e+00 - total_loss: 11046.2619


{'factorized_top_k/top_1_categorical_accuracy': 0.48961952328681946,
 'factorized_top_k/top_5_categorical_accuracy': 0.5723592638969421,
 'factorized_top_k/top_10_categorical_accuracy': 0.6231757998466492,
 'factorized_top_k/top_50_categorical_accuracy': 0.6944058537483215,
 'factorized_top_k/top_100_categorical_accuracy': 0.7076963186264038,
 'loss': 7286.86328125,
 'regularization_loss': 0,
 'total_loss': 7286.86328125}

In [46]:
# recommending Top-10 products for customer 52228204

# Create a item_item_model_4 that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(item_item_model_4.product_model)
# recommends products out of the entire products dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((products.batch(100), products.batch(100).map(item_item_model_4.product_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["52228204"]))
print(f"Recommendations for user 52228204: {titles[0, :10]}")

Recommendations for user 52228204: [b'Pelican Deluxe Marine Seat Cushion Cooler'
 b'i-smile Replacement Bands with Metal Clasps for Fitbit Flex, Set of 3 with 2 Piece Silicon Fastener Ring'
 b'i-smile Replacement Bands with Metal Clasps for Fitbit Flex, Set of 3 with 2 Piece Silicon Fastener Ring'
 b'Rough Rack 3-6 Ski & Snowboard Rack'
 b'Rough Rack 3-6 Ski & Snowboard Rack'
 b'AO Coolers NU-ICE Ice Enhancer,  5 lbs'
 b"Royal Robbins Men's Vernon Short Sleeve Top"
 b'Evo Sync AM/FM Handlebar Radio, With LED And Horn'
 b'Dragon Alliance Teal/Green Ion Cinch Jet Sunglasses'
 b"Currie Technologies Women's eZip Trailz Commuter Lithium Electric Bicycle, White, 15-Inch"]


Even though the second fine-tuned model performs slightly better than the first one on the test data: Top-10 AR is at 62% vs 59%, the recommendation's outputs has repetions and some suggestions are off. We will use first fine-tuned item-to-item model as a final model and run this model on the full dataset. 